This script will fetch locations of all venues in the Austin area that match a certain criteria (coffe, bar, bubble tea etc.).

In [4]:
import requests
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt

Since foursquare queries cannot return more than 50 venues at a time we have to run multiple scans to cover all the area.
To do this I will divide the area into square 1kmx1km cells.
1 KM is equal to 1/111 degrees. 

In [5]:
km=1/111
lat_A=30.2672
lng_A=-97.7431
lat=lat_A-km*7
lng=lng_A-km*10
n=25

In [6]:
CLIENT_ID = 'SVC2EATH3GGQLSZVIQFAW1PH0O2RDX5K0H30ZJ3LOBKRR4SK' # your Foursquare ID
CLIENT_SECRET = '1A0VXT2XCTZ10JBCRLFKYNJDDHLTUHNIM43TTNY5WJ1DISL0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SVC2EATH3GGQLSZVIQFAW1PH0O2RDX5K0H30ZJ3LOBKRR4SK
CLIENT_SECRET:1A0VXT2XCTZ10JBCRLFKYNJDDHLTUHNIM43TTNY5WJ1DISL0


Initialise the database with 4 columns: id of venue, name and geo coordinates.

In [9]:
col=['i

In [9]:
d','name','location.lat','location.lng']
db  = pd.DataFrame(columns = col)
db.head()
db.shape

(0, 4)

Run the search query with raduis of sqrt(2)*1KM for every node of the square grid. Save the information to db dataframe

In [ ]:
radius = 1000*np.sqrt(2)

for i in range(n):
    for j in range(n):
        
        lat1=lat+km*i
        lng1=lng+km*j
        print('Currently scanning cell:',i,j,'Total of venues:',db.shape)
        search_query = 'coffee'
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat1, lng1, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
        from pandas.io.json import json_normalize
        venues = results['response']['venues']
        dataframe = json_normalize(venues)
        if venues!=[]:
            dataframe.head()
            df=dataframe[['id','name','location.lat','location.lng']].copy()
            frames = [db, df]
            db = pd.concat(frames)

Since there is a lot of coffee shops in the area that I selected I exceeded the quota for one day, so I had to finish the scan the next day.
The compound dataframe was saved to coffeeshops CSV file.

In [16]:
db.to_csv('coffeeshops2.csv')

In [15]:
db.shape
# at [18,18] quota exceeded
# continue the following day

(1906, 4)

In [21]:
Austin=folium.Map(location=[lat_A, lng_A], zoom_start=11)



for i in range(n):
    for j in range(n):
        folium.CircleMarker(
        [lat+km*i, lng+km*j],
        radius=1,
        color='black',
        fill=False,
        parse_html=False).add_to(Austin)  
        
for lat, lng in zip(db['location.lat'],db['location.lng']):
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        color='red',
        fill=False,
        parse_html=False).add_to(Austin)

The following map shows the grid of scan centers(black) and coffeshop locations(red). There is 25x25=625 search queries.
GitHub won't show the map. It can be accessed here: http://nbviewer.jupyter.org/github/drpurtseladze/capstone/blob/master/cap2a.ipynb

In [22]:
Austin